In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.6.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.21.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.6.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cache


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import os


In [11]:
if torch.cuda.is_available():
    print("✅ GPU Available:", torch.cuda.get_device_name(0))
else:
    print("❌ GPU not available. Check CUDA installation.")



✅ GPU Available: NVIDIA GeForce RTX 3050 Laptop GPU


In [12]:
model_id = "runwayml/stable-diffusion-v1-5"

# Load model using float16 for lower memory and faster inference
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)

pipe.to("cuda")  # Move model to GPU
pipe.enable_attention_slicing()  # Reduce memory load

print("🚀 Model loaded successfully to GPU")


Loading pipeline components...: 100%|██████████| 7/7 [00:04<00:00,  1.70it/s]


🚀 Model loaded successfully to GPU


In [23]:
def generate_image(prompt, output_path="generated.png", steps=50, scale=9.0, seed=42):
    generator = torch.Generator("cuda").manual_seed(seed)

    image = pipe(
        prompt,
        num_inference_steps=steps,
        guidance_scale=scale,
        generator=generator
    ).images[0]

    image.save(output_path)

    torch.cuda.empty_cache()  # Clean up
    return image


In [28]:
prompt = "a skyscraper with 3 tone color pallet, realistic"
output_path = "generated.png"

img = generate_image(prompt, output_path, steps=60, scale=10)
img.show()


100%|██████████| 60/60 [00:17<00:00,  3.38it/s]
